#1. Saving the model during development using checkpoints

In [1]:
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.11.0


In [3]:
# Create a sample dataset 

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# Define a simple model

# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt" # Provide the checkpoint path here
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 1.1555 - sparse_categorical_accuracy: 0.6660
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 3s 46ms/step - loss: 1.1555 - sparse_categorical_accuracy: 0.6660 - val_loss: 0.7164 - val_sparse_categorical_accuracy: 0.7920
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4201 - sparse_categorical_accuracy: 0.8810
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 26ms/step - loss: 0.4204 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.5337 - val_sparse_categorical_accuracy: 0.8380
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.2910 - sparse_categorical_accuracy: 0.9180
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 32ms/step - loss: 0.2910 - sparse_categorical_accuracy: 0.9180 - val_loss: 0.4543 - val_sparse_categorical_accuracy: 0.8580
Epoch 4/10
32/32 [=======

In [6]:
os.listdir(checkpoint_dir) # To look at what files got generated

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

## What are these files?
---
The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

One or more shards that contain your model's weights.
An index file that indicates which weights are stored in which shard.
If you are training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001

#2. Loading the model from the saved checkpoints


In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 1s - loss: 2.3298 - sparse_categorical_accuracy: 0.1440 - 686ms/epoch - 21ms/step
Untrained model, accuracy: 14.40%


In [8]:
# Callback checkpoints
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [9]:
# To obtain the latest checkpoint of training
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [10]:
# To load the model with latest checkpoint weights

# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4734 - sparse_categorical_accuracy: 0.8800 - 382ms/epoch - 12ms/step
Restored model, accuracy: 88.00%


#3. Manually Saving Weights & Model Configuration Separately

In [11]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc)) 

32/32 - 0s - loss: 0.4734 - sparse_categorical_accuracy: 0.8800 - 370ms/epoch - 12ms/step
Restored model, accuracy: 88.00%


#4. Saving Complete Model

In [12]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 2s 12ms/step - loss: 1.1493 - sparse_categorical_accuracy: 0.6740
Epoch 2/5
32/32 [==============================] - 0s 13ms/step - loss: 0.4301 - sparse_categorical_accuracy: 0.8860
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2852 - sparse_categorical_accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 9ms/step - loss: 0.2092 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 9ms/step - loss: 0.1552 - sparse_categorical_accuracy: 0.9620


In [13]:
# Loading from the saved model
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#5. Saving & Loading model in HDF5 Format

In [15]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Epoch 1/5
32/32 [==============================] - 2s 16ms/step - loss: 1.1748 - sparse_categorical_accuracy: 0.6500
Epoch 2/5
32/32 [==============================] - 1s 16ms/step - loss: 0.4418 - sparse_categorical_accuracy: 0.8690
Epoch 3/5
32/32 [==============================] - 0s 14ms/step - loss: 0.3022 - sparse_categorical_accuracy: 0.9090
Epoch 4/5
32/32 [==============================] - 0s 14ms/step - loss: 0.2114 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 1s 17ms/step - loss: 0.1496 - sparse_categorical_accuracy: 0.9660
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                               